In [5]:
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm
from langchain.chat_models import ChatOpenAI

chat = load_llm("model.json")

/opt/homebrew/anaconda3/envs/gpt_project/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/anaconda3/envs/gpt_project/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [6]:
chat

OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})